<a href="https://colab.research.google.com/github/candidowe/DIO_Tradutor/blob/main/traduto_dio_artigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4 openai langchain-openai


In [4]:
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from langchain_openai.chat_models.azure import AzureChatOpenAI

# --- Carrega as variáveis de ambiente do config.env ---
load_dotenv("config.env")

# Lê as variáveis do arquivo .env
AZURE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

# --- Função para extrair texto limpo de uma URL ---
def extract_text_from_url(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        for script_or_style in soup(["script", "style"]):
            script_or_style.decompose()

        text = soup.get_text(separator=' ')
        linhas = (line.strip() for line in text.splitlines())
        part = (phrase.strip() for line in linhas for phrase in line.split("  "))
        texto_limpo = '\n'.join(part for part in part if part)
        return texto_limpo
    else:
        print(f"Falha ao acessar a página. Código: {response.status_code}")
        return None


# --- Inicializa o cliente Azure OpenAI ---
client = AzureChatOpenAI(
    azure_endpoint=AZURE_ENDPOINT,
    api_key=AZURE_KEY,
    api_version=AZURE_VERSION,
    deployment_name=AZURE_DEPLOYMENT,
    max_retries=0
)

# --- Função de tradução ---
def translate_article(text, lang):
    messages = [
        ("system", "Você é um tradutor de textos técnicos e artigos."),
        ("user", f"Traduza o seguinte texto para o idioma {lang} e mantenha a formatação em Markdown:\n\n{text}")
    ]
    response = client.invoke(messages)
    print(response.content)
    return response.content


# --- Exemplo de uso ---
if __name__ == "__main__":
    url = "https://dev.to/edgaras/ethereum-integration-in-php-for-wallets-and-transfers-11d2"
    article_text = extract_text_from_url(url)

    if article_text:
        translated = translate_article(article_text[:8000], "português")

        # Opcional: salvar tradução em arquivo .md
        with open("artigo_traduzido.md", "w", encoding="utf-8") as f:
            f.write(translated)
        print("\n✅ Tradução salva em 'artigo_traduzido.md'")


# Integração Ethereum em PHP para Carteiras e Transferências - DEV Community

## Edgaras  
Publicado em 6 de outubro

### Ethereum Integration in PHP for Wallets and Transfers  
`#php`  
`#ethereum`  
`#web3`  
`#blockchain`

Se você desenvolve backends em PHP e quer interagir com Ethereum sem precisar de toda uma stack Node completa, esta biblioteca oferece métodos simples para as funcionalidades principais: gerenciamento de carteiras, transferências de ETH (legacy e EIP-1559), saldos, consultas de transações e utilitários.

---

### GitHub:  
[Edgaras0x4E/Ethereum](https://github.com/Edgaras0x4E/Ethereum)

---

## O que você pode fazer

- **Carteiras**: Criar/importar carteiras, derivar endereço/chave pública  
- **Transferências**: Enviar ETH usando transações legacy (Tipo-0) e EIP-1559 (Tipo-2); aguardar confirmações  
- **Saldos e rede**:  
  - `getBalance`  
  - `getBalanceInEther`  
  - `getBlockNumber`  
  - `getNetworkInfo`  
- **Transações**:  
  - `getTransaction`  
  - `get